In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
torch.backends.cudnn.benchmark = True
import selfies
import json
import os
import pandas as pd
from tqdm import tqdm

from VAE import VAE_encode, VAE_decode

In [2]:
one_hot_path = '../csc-one-hot'

In [3]:
# load model parameters
with open('BO_use/settings.json') as f:
  settings = json.load(f)

encoder_parameter = settings['encoder']
latent_dimension = settings['training']['latent_dimension']

# load an instance to determine SELFIES-relaterd parameters
data = np.load(os.path.join(one_hot_path, '0.npy'))

len_max_molec = data.shape[0]
len_alphabet = data.shape[1]
len_max_molec1Hot = len_max_molec * len_alphabet

model_encode = VAE_encode(len_max_molec1Hot, **encoder_parameter)

epoch =  9032

encoder_state = torch.load('BO_use/%d/encode.tar' % epoch)

model_encode.load_state_dict(encoder_state['state_dict'])

model_encode.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

model_encode.to(device)

print('model ready!')

cuda
model ready!


In [4]:
# csc compound number: 147428
idx_list = np.arange(147428)
batch_size = 2000
latent_space= []
num_batches = int(len(idx_list) / batch_size) + 1

for batch_iteration in tqdm(range(num_batches)):

    current_idx_start = batch_iteration * batch_size
    current_idx_stop = (batch_iteration + 1) * batch_size
    if batch_iteration < num_batches - 1:
        idx_smile_hot = idx_list[current_idx_start: current_idx_stop]
    else:
        idx_smile_hot = idx_list[current_idx_start:]

    inp_smile_hot = np.array([np.load(os.path.join(one_hot_path, '{}.npy'.format(idx))) for idx in idx_smile_hot])
    inp_smile_hot = torch.tensor(inp_smile_hot, dtype=torch.float).to(device)

    inp_smile_encode = inp_smile_hot.reshape(inp_smile_hot.shape[0],
                                             inp_smile_hot.shape[1] * inp_smile_hot.shape[2])
    
    latent_points, mus, log_vars = model_encode(inp_smile_encode)
    latent_space.append(latent_points.cpu().detach().numpy())

latent_space = np.concatenate(latent_space)

100%|██████████| 74/74 [00:36<00:00,  2.01it/s]


In [5]:
latent_space.shape

(147428, 100)

In [6]:
np.save('csc-latent-vecs.npy', latent_space)